In [1]:
import numpy as np
import pandas as pd
import cvxopt as cvx
from cvxopt import matrix
from cvxopt import solvers

In [2]:
df = pd.read_csv('spambase.data',header=None).to_numpy()

In [3]:
#separate data and labels
X = df[:,0:57]
y = df[:,57]
#change labels to -1 and 1
y = 2*y - 1

In [4]:
#separate training and test data
split = .5
N = X.shape[0]
d = X.shape[1]
idx = np.random.choice(np.arange(N),size=int(np.floor(split*N)),replace=False)
test_idx = np.setdiff1d(np.arange(N), idx)
#split data
X_train = X[idx,:]
X_test = X[test_idx,:]
#split labels
y_train = y[idx]
y_test = y[test_idx]
#set up some constants
N_train = X_train.shape[0]
N_test = X_test.shape[0]

In [5]:
#standardize data for better results
mean = np.mean(X_train,axis=0)
std = np.std(X_train,axis=0)
X_train = 1/std*(X_train-mean)
X_test = 1/std*(X_test- mean)

In [6]:
#kernels
#Gaussian kernel
def K1(x,y,sigma=1):
    return np.exp(-1/(2*sigma**2)*np.linalg.norm(x-y,axis=1)**2)
#polynomial kernel
def K2(x,y,sigma=0,d=2):
    return (np.sum(x*y,axis=1)+sigma)**d
#linear kernel
def K3(x,y,sigma=0):
    return np.sum(x*y,axis=1)+sigma
#sigmoid kernel
def K4(x,y,sigma=-1,kappa=1):
    return np.tanh(kappa*np.sum(x*y,axis=1)+sigma)

In [7]:
#functions for gram matrices 
#Gaussian kernel GM
def gramK1(X,sigma=1):
    return np.exp(-1/(2*sigma**2)*np.linalg.norm(X.T-X[:,:,None],axis=1)**2)
#polynomial kernel gram matrix
def gramK2(X,sigma=0,d=2):
    return (np.sum(X.T*X[:,:,None],axis=1)+sigma)**d
#linear kernel GM
def gramK3(X,sigma=0):
    return np.sum(X.T*X[:,:,None],axis=1)+sigma
#sigmoid GM
def gramK4(X,sigma=-1,kappa=1):
    return np.tanh(kappa*np.sum(X.T*X[:,:,None],axis=1)+ sigma)

In [8]:
#kernel dictionary
kern_dict = {'gaussian':'K1','polynomial':'K2','linear':'K3','sigmoid':'K4'}
#gram matrix dictionary
GM_dict = {'gaussian':'gramK1','polynomial':'gramK2', 'linear':'gramK3','sigmoid':'gramK4'}

In [9]:
kernel = 'gramK1'
#specify kernel to use
GM = eval(kernel)
gram = GM(X_train)

In [10]:
#regularization hyperparameter
C = 100

In [11]:
#set up and solve SVM optimization problem
P = matrix(y_train[:,None]@y_train[None,:]*gram)
q = matrix(-np.ones(N_train))
G = matrix(np.concatenate([np.eye(N_train),-np.eye(N_train)],axis=0))
h = matrix(np.concatenate([C*(1/N_train)*np.ones(N_train),np.zeros(N_train)]))
A = matrix(y_train[None,:])
b = matrix(np.zeros(1)[None,:])
sol = solvers.qp(P,q,G,h,A,b)
#get solution alpha
alpha = sol['x']

     pcost       dcost       gap    pres   dres
 0: -4.8977e+02 -3.4968e+02  2e+04  6e+01  2e-15
 1: -1.5767e+02 -2.5164e+02  1e+03  3e+00  1e-15
 2: -7.2611e+01 -1.8843e+02  1e+02  2e-15  4e-15
 3: -7.6784e+01 -8.8261e+01  1e+01  7e-16  2e-15
 4: -7.7593e+01 -8.1274e+01  4e+00  4e-16  9e-16
 5: -7.8013e+01 -7.8998e+01  1e+00  2e-15  1e-15
 6: -7.8188e+01 -7.8419e+01  2e-01  6e-16  1e-15
 7: -7.8225e+01 -7.8318e+01  9e-02  3e-16  1e-15
 8: -7.8240e+01 -7.8280e+01  4e-02  4e-16  1e-15
 9: -7.8248e+01 -7.8258e+01  1e-02  2e-15  1e-15
10: -7.8249e+01 -7.8256e+01  8e-03  8e-16  7e-16
11: -7.8251e+01 -7.8252e+01  2e-03  1e-15  1e-15
12: -7.8251e+01 -7.8252e+01  9e-04  3e-15  9e-16
13: -7.8251e+01 -7.8251e+01  3e-04  7e-16  1e-15
14: -7.8251e+01 -7.8251e+01  3e-05  6e-16  1e-15
Optimal solution found.


In [12]:
alpha = np.array(alpha)

In [19]:
def predict(x,X,alpha,y,K):
    x_rep = np.repeat(x,N_train,axis=0)
    kern_vec = K(x_rep,X)
    val = alpha.T@(y*kern_vec)
    if val > 0:
        return 1
    else:
        return -1

In [20]:
K = eval('K1')
num_correct = 0
for i in range(N_test):
    pred = predict(X_test[None,i,:],X_train,alpha,y_train,K)
    if y_test[i] == pred:
        num_correct = num_correct + 1
100*num_correct/N_test

90.8735332464146